# Emotions Episodes Votes

In [10]:
import pandas as pd

file_path = './raw_data/emotions_episode_votes.csv'
file_path_new = './data/emotions_episode_votes_modified.csv'

# Attempt to correctly parse the CSV file by detecting the delimiter automatically
df_new_corrected = pd.read_csv(file_path, sep=None, engine='python')

# Convert the column names to lowercase
df_new_corrected.columns = map(str.lower, df_new_corrected.columns)

# Retain only the specified columns
columns_to_keep_corrected = ['episode_id', 'emotion_id', 'user_id', 'created']
df_new_filtered = df_new_corrected[columns_to_keep_corrected]

# Display the modified dataframe
df_new_filtered.head()

# # Save the modified dataframe to a new CSV file
# df_new_filtered.to_csv(file_path_new, index=False)

,episode_id,emotion_id,user_id,created
0,9279985,33,22659137,2023-09-04 12:31:12 +0000 UTC
1,8355786,34,22659137,2023-01-28 21:45:44 +0000 UTC
2,8987962,31,22659137,2023-05-10 19:18:18 +0000 UTC
3,9333705,30,22659137,2022-10-08 12:51:28 +0000 UTC
4,9138258,30,22659137,2022-11-17 11:56:07 +0000 UTC


# Episode Comments

In [11]:
import pandas as pd

file_path = './raw_data/episode_comment.csv'
file_path_new = './data/episode_comment_modified.csv'

# Load the dataset
df = pd.read_csv(file_path, delimiter=',')

# Filter the rows where depth is greater than 0 - we don't want the replies to the comments
df_filtered = df[df['depth'] < 1]

# Select only specified columns
df_filtered = df_filtered[['episode_id', 'tv_show_name', 'episode_season_number', 'episode_number', 'user_id', 'comment', 'nb_likes', 'created_at', 'updated_at']]


# Save the modified dataframe to a new CSV file
df_filtered.to_csv(file_path_new, index=False)
df_filtered.head()

,episode_id,tv_show_name,episode_season_number,episode_number,user_id,comment,nb_likes,created_at,updated_at
1,9247265,Cobra Kai,5,7,22659137,"Wow, what a surprise ! The plan of silver is t...",0,2022-10-06 13:21:54,2022-10-06 13:21:55
2,9333705,Monster (2022),1,2,22659137,It’s so frustrating to see how easy it could h...,4,2022-10-08 12:54:38,2022-12-01 13:33:06
3,6104951,Ozark,1,1,22659137,Never too late to start a good show !,0,2022-10-09 14:04:44,2022-10-09 14:04:47
4,6104952,Ozark,1,2,22659137,Better call Marty,5,2022-10-09 21:00:12,2023-04-11 11:40:31
5,6172297,Ozark,1,9,22659137,That’s rare to have a show with smarts and log...,0,2022-10-11 21:12:53,2022-10-11 21:12:53


# Followed TV Show preprocessing

In [12]:
import pandas as pd

file_path = './raw_data/followed_tv_show.csv'
file_path_new = './data/followed_tv_show_modified.csv'

# Attempt to correctly parse the CSV file by detecting the delimiter automatically
df_new_corrected = pd.read_csv(file_path, sep=None, engine='python')

# Display the corrected dataframe structure to verify the column names and data
df_new_corrected.head()

# Retain only the specified columns
columns_to_keep_corrected = ['tv_show_name', 'tv_show_id', 'created_at', 'active', 'archived', 'user_id']
df_new_filtered = df_new_corrected[columns_to_keep_corrected]


# Save the modified dataframe to a new CSV file
df_new_filtered.to_csv(file_path_new, index=False)
df_new_filtered.head()

,tv_show_name,tv_show_id,created_at,active,archived,user_id
0,My Wife and Kids,70329,2022-12-26 15:48:36,1,0,22659137
1,The Simpsons,71663,2018-08-11 03:39:56,1,0,22659137
2,American Dad!,73141,2018-08-11 03:39:56,1,0,22659137
3,The Office (US),73244,2020-12-19 23:16:47,1,0,22659137
4,Malcolm in the Middle,73838,2020-03-20 13:15:23,1,0,22659137


# Rating Episode preprocessing

In [13]:
# Define a mapping from order values to note values
order_mapping = {'1': 1, '27': 2, '28': 3, '29': 4, '3': 5}

# Function to convert order and RATING_ID to note
def convert_to_note(row):
    # Split the 'order' string into a list of strings, then convert to a list of integers
    order_values = list(map(int, row['order'].split(',')))
    # Get the index of RATING_ID in order_values, then use this index to find the corresponding note value
    try:
        rating_index = order_values.index(row['RATING_ID'])
        return order_mapping[str(order_values[rating_index])]
    except ValueError:
        # In case the RATING_ID is not found in the order list, return NaN or some error indicator
        return float('nan')

file_path = './raw_data/ratings_episode_votes.csv'
df = pd.read_csv(file_path, sep=None, engine='python')

# Apply the conversion function to each row
df['note'] = df.apply(convert_to_note, axis=1)

# Drop the specified columns
columns_to_drop = ['order', 'RATING_ID', 'VOTE_KEY', 'IS_DELETED', 'DB_UPDATE_TS', 'set']
df.drop(columns=columns_to_drop, inplace=True)
df.columns = map(str.lower, df.columns)

# Save the modified dataframe to a new CSV file
output_file_path = './data/ratings_episode_votes_modified.csv'
df.to_csv(output_file_path, index=False)
df.head()

,episode_id,user_id,created,note
0,9300505,22659137,2022-09-06 20:42:29 +0000 UTC,5
1,8791888,22659137,2022-10-31 13:33:25 +0000 UTC,5
2,8287133,22659137,2023-01-03 23:00:35 +0000 UTC,4
3,8868130,22659137,2022-11-24 23:27:49 +0000 UTC,5
4,7620099,22659137,2022-10-24 08:25:16 +0000 UTC,5


# Seen Episodes

In [14]:
# Load the datasets
rewatched_episode_df = pd.read_csv('./raw_data/rewatched_episode.csv')
seen_episode_df = pd.read_csv('./raw_data/seen_episode.csv')

# Drop updated_at of seen_episode_df
seen_episode_df.drop(columns=['updated_at', 'tweet_id'], inplace=True)
# Drop created_at of rewatched_episode_df
rewatched_episode_df.drop(columns=['created_at'], inplace=True)

# Merge the datasets on specified columns
merge_columns = ['tv_show_name', 'episode_season_number', 'episode_number', 'user_id', 'episode_id']
final_df = pd.merge(seen_episode_df, rewatched_episode_df[merge_columns + ['cpt']], 
                    on=merge_columns, 
                    how='left')

# Replace NaN in rewatched_count with 0
final_df['rewatched_count'] = final_df['cpt'].fillna(0).astype(int)
final_df.drop(columns=['cpt'], inplace=True)  # Drop the 'cpt' column as it's no longer needed

# Remove the rows where tv_show_name is NaN or empty
final_df = final_df[final_df['tv_show_name'].notna()]

# # Select and rename relevant columns for the final dataset
# final_dataset = final_df[['updated_at', 'tv_show_name', 'episode_season_number', 'episode_number', 
#                           'user_id', 'episode_id', 'created_at', 'rewatched_count']]


# Save the final dataset to a new CSV file
file_path_new = './data/seen_episode_modified.csv'
final_df.to_csv(file_path_new, index=False)
final_df.head()

,episode_id,created_at,fb_action_id,user_id,tv_show_name,episode_season_number,episode_number,rewatched_count
0,55452,2018-08-11 03:39:56,NaN,22659137,The Simpsons,1,1,0
1,55453,2018-08-11 03:39:56,NaN,22659137,The Simpsons,1,2,0
2,55454,2018-08-11 03:39:56,NaN,22659137,The Simpsons,1,3,0
3,55455,2018-08-11 03:39:56,NaN,22659137,The Simpsons,1,4,0
4,55456,2018-08-11 03:39:56,NaN,22659137,The Simpsons,1,5,0
